# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

**Признаки**  
`DateCrawled` — дата скачивания анкеты из базы  
`VehicleType` — тип автомобильного кузова  
`RegistrationYear` — год регистрации автомобиля  
`Gearbox` — тип коробки передач  
`Power` — мощность (л. с.)  
`Model` — модель автомобиля  
`Kilometer` — пробег (км)  
`RegistrationMonth` — месяц регистрации автомобиля  
`FuelType` — тип топлива  
`Brand` — марка автомобиля  
`Repaired` — была машина в ремонте или нет  
`DateCreated` — дата создания анкеты  
`NumberOfPictures` — количество фотографий автомобиля  
`PostalCode` — почтовый индекс владельца анкеты (пользователя)  
`LastSeen` — дата последней активности пользователя  

**Целевой признак**  
`Price` — цена (евро)

## Подготовка данных

In [1]:
!pip install scikit-learn==1.1.3
!pip install lightgbm

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
from time import time
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [3]:
data = pd.read_csv('/datasets/autos.csv')
display(data.info())
data.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

None

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


Удалим явные дубликаты

In [4]:
data = data.drop_duplicates()

In [5]:
data.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354365.000000,354365.000000,354365.000000,354365.000000,354365.000000,354365.0,354365.000000
mean,4416.679830,2004.234481,110.093816,128211.363989,5.714650,0.0,50508.503800
std,4514.176349,90.228466,189.851330,37905.083858,3.726432,0.0,25783.100078
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


1) В некоторых признаках присутствуют пропуска;  
2) Столбцы с датами имеют неправильный тип;  Однако сами признаки-даты скорее всего не влияют на стоимость автомобилей, поэтому от них можно избавиться.  
3) В столбце `NumberOfPictures` одни нули, это не несет никакой информации, от столбца нужно избавиться;  
4) Столбец с почтовым кодом `PostalCode` в нашем случае бесполезен, но потенциально с его помощью можно получить информацию о регионе продавца. Удалим столбец;  
5) Некоторые столбцы с числовым типом данных имеют явные выбросы и аномалии (нулевая стоимость, 9999й год регистрации,  
   мощность  20000 л.с. и тому подобные)


Избавимся от аномальных значений в столбцах `Price`, `RegistrationYear` и `Power`:  
1) В `Price` найдем цены меньше 10 у.е. (отсечем равные нулю, т.е. явно аномальные цены). Удалим эти записи, т.к. данный признак является целевым.     
2) В `Power` заменим аномальные значения медианным значением.  
3) В `RegistrationYear` заменим неадекватные года медианным годом.

Преобразуем столбец с датой скачивания анкеты `DateCrawled` в соответствующий дате тип `datetime`.  Затем извлечем из даты год; это понадобится, чтобы избавиться от аномальных значений в столбце с датой регистрации `RegistrationYear` (год регистрации автомобился не может быть больше года загрузки объявления).

In [6]:
data['DateCrawled'] = pd.to_datetime(data['DateCrawled'], format='%Y-%m-%d %H:%M:%S')
data['DateCrawled'] = pd.DatetimeIndex(data['DateCrawled']).year

In [7]:
data = data[data['Price'] >= 10]

data['Power'].mask((data['Power'] < 10) | (data['Power'] >= 1500), data['Power'].median(), inplace=True)

data.loc[(data['RegistrationYear'] > data['DateCrawled']) | (data['RegistrationYear'] < 1900), 
         'RegistrationYear'] = data['RegistrationYear'].median()

Удалим столбцы с датами - `DateCrawled`, `DateCreated`, `LastSeen`, `RegistrationMonth`.  

In [8]:
cols_to_drop = ['DateCrawled', 'DateCreated', 'LastSeen', 'RegistrationMonth']
data = data.drop(cols_to_drop, axis=1)

Избавимся от неинформативных столбцов `PostalCode` и `NumberOfPictures`. 

In [9]:
data = data.drop(['PostalCode', 'NumberOfPictures'], axis=1)

In [10]:
data.describe()

,Price,RegistrationYear,Power,Kilometer
count,342337.000000,342337.000000,342337.000000,342337.000000
mean,4571.855254,2002.624817,119.085173,128387.276280
std,4514.902138,6.753084,54.962918,37422.793501
min,10.000000,1910.000000,10.000000,5000.000000
25%,1200.000000,1999.000000,83.000000,125000.000000
50%,2900.000000,2003.000000,105.000000,150000.000000
75%,6500.000000,2007.000000,143.000000,150000.000000
max,20000.000000,2016.000000,1436.000000,150000.000000


От неинформативных признаков избавились, аномальные значения убрали.

Рассмотрим категориальные признаки.  
Вызывают сомнения признаки с большим числом категорий:

In [11]:
data['Brand'].nunique()

40

In [12]:
data['Model'].nunique()

250

Признак `Model` предлагю исключить ввиду слишком большого количества категорий 

In [13]:
data = data.drop(['Model'], axis=1)

Взглянем на итоговый датасет:

In [14]:
data.head()

,Price,VehicleType,RegistrationYear,Gearbox,Power,Kilometer,FuelType,Brand,Repaired
0,480,NaN,1993.0,manual,105,150000,petrol,volkswagen,NaN
1,18300,coupe,2011.0,manual,190,125000,gasoline,audi,yes
2,9800,suv,2004.0,auto,163,125000,gasoline,jeep,NaN
3,1500,small,2001.0,manual,75,150000,petrol,volkswagen,no
4,3600,small,2008.0,manual,69,90000,gasoline,skoda,no


Данные готовы для разделения на выборки и дальнейшего обучения моделей.

## Обучение моделей

Подготовим данные для обучения и тестирования моделей

In [15]:
# Отберем категориальные признаки, подлежащие кодированию
ohe_features = data.select_dtypes(include='object').columns.to_list()
print(ohe_features)

['VehicleType', 'Gearbox', 'FuelType', 'Brand', 'Repaired']


In [16]:
# Разобьем датасеты на тренировочную, валидационную и тестовую выборки в соотношении 60:20:20
data_train, data_buff = train_test_split(data, test_size=0.4, random_state=12345)
data_test, data_valid = train_test_split(data_buff, test_size=0.5, random_state=12345)

features_train = data_train.drop(['Price'], axis=1)
target_train = data_train['Price']

features_valid = data_valid.drop(['Price'], axis=1)
target_valid = data_valid['Price']

features_test = data_test.drop(['Price'], axis=1)
target_test = data_test['Price']

# Закодируем категориальные признаки методом One Hot Encoding
encoder = OneHotEncoder(drop='first', handle_unknown='ignore', sparse=False)
encoder.fit(features_train[ohe_features])

features_train[encoder.get_feature_names_out()] = encoder.transform(features_train[ohe_features])
features_train = features_train.drop(ohe_features, axis=1)

features_valid[encoder.get_feature_names_out()] = encoder.transform(features_valid[ohe_features])
features_valid = features_valid.drop(ohe_features, axis=1)

features_test[encoder.get_feature_names_out()] = encoder.transform(features_test[ohe_features])
features_test = features_test.drop(ohe_features, axis=1)

Теперь рассмотрим несколько моделей и подберем для них оптимальные параметры

### Модель дерева решений

In [17]:
best_score = 100000
best_depth_tree = 0
best_time = 0

for depth in range(1,7):
    time_start = time()

    model = DecisionTreeRegressor(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    score = (mean_squared_error(target_valid, predicted_valid))**0.5
    
    time_finish = time()
    time_final = time_finish - time_start
    
    if score < best_score:
        best_time = time_final        
        best_score = score
        best_depth_tree = depth

        
print(f'Лучшая RMSE равна {best_score:.3f}, достигается при глубине дерева {best_depth_tree}')
print(f'Время обучения составляет {best_time:.2f} сек.')

Лучшая RMSE равна 2336.053, достигается при глубине дерева 6
Время обучения составляет 0.75 сек.


### Модель случайного леса

*Примечание*: диапазон перебираемых параметров сделан небольшим ради экономии времени.

In [18]:
best_est_forest = 0
best_score = 100000
best_depth_forest = 0
best_time = 0

for depth in range(3,7): 
    for est in range(15, 30, 5):
        
        time_start = time()
        
        model = RandomForestRegressor(random_state=12345, n_estimators = est, max_depth=depth)
        model.fit(features_train, target_train)
        predicted_valid = model.predict(features_valid)
        score = (mean_squared_error(target_valid, predicted_valid))**0.5
        
        time_finish = time()
        time_final = time_finish - time_start
        
        if score < best_score:
            best_time = time_final
            best_score = score
            best_est_forest = est
            best_depth_forest = depth
            
print(f'Лучшая RMSE равна {best_score:.3f}, достигается при глубине дерева {best_depth_forest} и количестве деревьев {best_est_forest}')
print(f'Время обучения составляет {best_time:.2f} сек.')

Лучшая RMSE равна 2300.116, достигается при глубине дерева 6 и количестве деревьев 25
Время обучения составляет 11.37 сек.


### Модель линейной регрессии

In [19]:
time_start = time()

model = LinearRegression()
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
score = (mean_squared_error(target_valid, predicted_valid))**0.5
            
time_finish = time()
time_final = time_finish - time_start
    
print(f'RMSE равна {score:.3f}')
print(f'Время обучения составляет {time_final:.2f} сек.')

RMSE равна 2876.802
Время обучения составляет 3.47 сек.


Модель линейной регрессии не подходит для работы с данным набором данных. Скорее всего, причиной является слишком большое количество категориальных признаков

### Градиентный бустинг

Рассмотрим следующие гиперпараметры и их влияние на качество модели:  
* `num_itearations` - количество итерраций (количество деревьев в ансамбле)
* `learning_rate` - скорость обучения

In [20]:
train_dataset = lgb.Dataset(features_train, target_train, feature_name=list(features_train))
valid_dataset = lgb.Dataset(features_valid, target_valid, feature_name=list(features_valid))

In [21]:
for lrate in [0.1, 0.5, 0.9]:
    for num_iters in [50, 100, 200]:
        time_start = time()
        
        params = {"objective": "regression", 
                  "metric":"rmse", 
                  "learning_rate":lrate, 
                  "verbose":-100}
        
        bst = lgb.train(params, train_set=train_dataset,  valid_sets=(valid_dataset,), 
                        num_boost_round=num_iters, callbacks=[lgb.early_stopping(stopping_rounds=5, verbose=False)])
        
        time_finish = time()
        time_final = time_finish - time_start
        print(f"\nLearning_rate {lrate}, Num_iterations {num_iters}, Время обучения {time_final:.2f} сек.")
        print('RMSE', bst.best_score["valid_0"]['rmse'])       



Learning_rate 0.1, Num_iterations 50, Время обучения 3.40 сек.
RMSE 1863.052503682442

Learning_rate 0.1, Num_iterations 100, Время обучения 5.01 сек.
RMSE 1781.7714470883902

Learning_rate 0.1, Num_iterations 200, Время обучения 13.29 сек.
RMSE 1733.112614992745

Learning_rate 0.5, Num_iterations 50, Время обучения 2.20 сек.
RMSE 1758.7875498245032

Learning_rate 0.5, Num_iterations 100, Время обучения 3.40 сек.
RMSE 1715.085080710197

Learning_rate 0.5, Num_iterations 200, Время обучения 6.20 сек.
RMSE 1686.3829654302472

Learning_rate 0.9, Num_iterations 50, Время обучения 1.91 сек.
RMSE 1773.4162194116257

Learning_rate 0.9, Num_iterations 100, Время обучения 2.60 сек.
RMSE 1762.2786661522348

Learning_rate 0.9, Num_iterations 200, Время обучения 2.40 сек.
RMSE 1762.2786661522348


В качестве оптимальных по соотношения скорость/качество примем следующие значения гиперпараметров:  
`learning_rate = 0.5`, `num_iterations = 100`

## Анализ моделей

Было рассмотрено несколько моделей - решающего дерева, случайного леса, линейной регрессии и градиентного бустинга.  
Модель случайного леса обучалась на порядок дольше, чем все остальные, однако результат оказался лишь немного лучше, чем у решающего дерева. Модель гредиентного бустинга показала лучший результат при небольшом времени обучения.  

В результате была выбрана модель градиентного бустинга из библиотеки LightGBM (`learning_rate = 0.5`, `num_iterations = 100`).  
Проверим эту модель на тестовой выборке.

|  Модель               |   RMSE        | Скорость обучения                      |
| :-------------------- | :---------------------: |:---------------------------:|
| Решающее дерево | 2336 | 0,73 с. |
| Случайный лес | 2300 | 11,9 с. |
| Градиентный бустинг (LightGBM) | 1715 | 5,2 с. |
| Линейная регрессия | 2876 | 3,5 с.|

## Тестирование лучшей модели

In [22]:
test_dataset = lgb.Dataset(features_test, target_test, feature_name=list(features_valid))

In [23]:
params = {"objective": "regression", 
            "metric":"rmse", 
            "learning_rate":0.5, 
            "num_iterations":100}
        
bst = lgb.train(params, train_set=train_dataset,  valid_sets=(valid_dataset,), 
                callbacks=[lgb.early_stopping(stopping_rounds=5, verbose=False)])

time_start = time()

predicted = bst.predict(features_test)

time_finish = time()
time_final = time_finish - time_start
print(f'\nRMSE {(mean_squared_error(target_test, predicted))**0.5:.2f}')
print(f'Время предсказания составляет {time_final:.2f} сек.')

/opt/conda/lib/python3.9/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005093 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 462
[LightGBM] [Info] Number of data points in the train set: 205402, number of used features: 61
[LightGBM] [Info] Start training from score 4569.364451

RMSE 1744.91
Время предсказания составляет 0.50 сек.


Модель показывает результат, отвечающий требованиям поставленной задачи (RMSE должно быть меньше 2500).  
Скорость предсказания менее 1 секунды.

## Вывод

Была проведена работа по исследованию базы данных с объявлениями о продаже машин из сервиса "Не бит, не крашен".  
Основной целью исследования была разработка модели для оценки стоимости автомобиля на основе его характеристик.  
Перед созданием модели данные были подготовлены следующим образом:  
* Удалены неинформативные признаки;  
* Удалены аномальные значения и выбросы.  

С финальным списком признаков, на основе которых производилась оценка стоимости, можно ознакомиться в конце Главы 1.  

Т.к. для заказчика важны скорость обучения и качество прогнозирования, было рассмотрено несколько моделей машинного обучения - линейная регрессия, решающее дерево, случайный лес и градиентный бустинг. В роли метрики качества была использована метрика RMSE.  
С результатом анализа моделей можно познакомиться в Главе 3.  

В итоге, наиболее подходящей моделью была выбрана модель на основе градиентного бустинга. Она показала хороший результат - оптимальное соотношение метрики качества и скорости предсказания и обучения.